## Learning Lyapunov function for Inverted Pendulum

In [2]:
# -*- coding: utf-8 -*-
# from dreal import *
from Functions import *
import torch 
import torch.nn.functional as F
import numpy as np
import timeit 
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os

import gymnasium as gym
from gym.envs.classic_control import PendulumEnv
from stable_baselines3 import DDPG
from stable_baselines3.common.env_util import DummyVecEnv
# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\dell\anaconda3\envs\tf115\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Neural network model
Building NN with random parameters for Lyapunov function and initializing parameters of NN controller to LQR solution

LQR solution is obtained by minimizing the cost function J = ∫(xᵀQx + uᵀRu)dt, where Q is 2×2 identity matrix and R is 1×1 identity matrix

In [3]:
class Net(torch.nn.Module):
    
    def __init__(self,n_input,n_hidden,n_output):
        super(Net, self).__init__()
        torch.manual_seed(2)
        self.layer1 = nn.Linear(n_input, n_hidden)
        self.layer2 = nn.Linear(n_hidden,n_output)
        # self.control = torch.nn.Linear(n_input,1,bias=False)
        # self.control.weight = torch.nn.Parameter(lqr)

    def forward(self,x):
        sigmoid = nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = sigmoid(self.layer2(h_1))
        # u = self.control(x)
        return out
    
class NeuralNetwork(nn.Module):
    def __init__(self, input_size=2, output_size=1):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # Input size: 3 (3D float input), Output size: 64
        self.fc2 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, output_size)  # Output size: 1 (1D float output)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc6(x)
        return x

## Policy model

In [4]:
# Create the Pendulum-v1 environment
env = PendulumEnv()
policy_model = DDPG.load("ddpg_pendulum_10k")

## Dynamical system

In [5]:
dynamic_model = NeuralNetwork(3,2)
dynamic_model.load_state_dict(torch.load('model_2_NN2.pth'))

<All keys matched successfully>

## Options

In [24]:
'''
For learning 
'''
N = 500             # sample size
N_test = 10
D_in = 2            # input dimension
H1 = 6              # hidden dimension
D_out = 1           # output dimension
torch.manual_seed(10)
# x_ = torch.Tensor(N, D_in).uniform_(-6, 6)
x1 = torch.Tensor(N, 1).uniform_(-np.pi, np.pi)
x2 = torch.Tensor(N, 1).uniform_(-8,8)
x = torch.cat((x1, x2), dim=1)
x_0 = torch.zeros([1, 2])

epsilon = 0
# Checking candidate V within a ball around the origin (ball_lb ≤ sqrt(∑xᵢ²) ≤ ball_ub)
ball_lb = 0.5
ball_ub = 74

## Learning and Falsification

In [25]:
out_iters = 0
valid = False
while out_iters < 2 and not valid: 
    start = timeit.default_timer()
    # lqr = torch.tensor([[-23.58639732,  -5.31421063]])    # lqr solution
    model = Net(D_in,H1, D_out)
    L = []
    i = 0 
    t = 0
    max_iters = 2000
    learning_rate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    while i < max_iters and not valid: 
        V_candidate = model(x)
        X0 = model(x_0)
        x_obs = torch.cat((np.cos(x[:,0]).reshape(-1,1), np.sin(x[:,0]).reshape(-1,1), x[:,1].reshape(-1,1)), dim=1)
        u, _ = policy_model.predict(x_obs)
        # print(type(x), type(u), type(torch.cat((x,torch.from_numpy(u)), dim=1)))
        f = dynamic_model(torch.cat((x,torch.from_numpy(u)), dim=1))
        # f = f_value(x,u)
        Circle_Tuning = Tune(x)
        # Compute lie derivative of V : L_V = ∑∂V/∂xᵢ*fᵢ
        L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()),0)

        # With tuning term 
        Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()\
                    +2.2*((Circle_Tuning-6*V_candidate).pow(2)).mean()+(X0).pow(2)
        # Without tuning term
#         Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()+ 1.2*(X0).pow(2)
        
        
        # print(i, "Lyapunov Risk=",Lyapunov_risk.item())
        L.append(Lyapunov_risk.item())
        optimizer.zero_grad()
        Lyapunov_risk.backward()
        optimizer.step()

        # Falsification
        if i % 100 == 0:

            print(i, "Lyapunov Risk=",Lyapunov_risk.item())
            x1_test = torch.Tensor(N_test, 1).uniform_(-np.pi, np.pi)
            x2_test = torch.Tensor(N_test, 1).uniform_(-8,8)
            x_test = torch.cat((x1_test, x2_test), dim=1)
            x_obs_test = torch.cat((np.cos(x1_test), np.sin(x1_test), x2_test), dim=1)

            V_candidate = model(x_test)
            u, _ = policy_model.predict(x_obs_test)
            f = dynamic_model(torch.cat((x_test,torch.from_numpy(u)), dim=1))
            # Compute lie derivative of V : L_V = ∑∂V/∂xᵢ*fᵢ
            L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x_test,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()),0)
            index = np.logical_and(V_candidate.reshape(-1)>0, L_V<epsilon)
            print("V:",V_candidate.detach().numpy().reshape(-1),'\nL_V:' , L_V.detach().numpy())          

            # print('===========Verifying==========')
            start_ = timeit.default_timer()
            result= CheckLyapunov(x_test, f, V_candidate.reshape(-1), L_V, ball_lb, ball_ub, epsilon)
            stop_ = timeit.default_timer()

            print("unsatisfied sample number:",result.size()[0],"/",N_test)
            if (result.size()[0]!=0): 
                # print("Not a Lyapunov function. Found counterexample: ")
                # print(result)
                # x = AddCounterexamples(x,result,10)
                pass
            else:  
                valid = True
                print("Satisfy conditions!!")
                print(V_candidate, " is a Lyapunov function.")
            t += (stop_ - start_)
            # print('==============================')
        i += 1

    stop = timeit.default_timer()


    torch.save(model.state_dict(), 'Lyapunov_NN_gym.pth')
    print('\n')
    print("Total time: ", stop - start)
    print("Verified time: ", t)
    
    out_iters+=1

0 Lyapunov Risk= 51.195308685302734
V: [0.06327085 0.07257738 0.1285501  0.13085963 0.12798062 0.11326627
 0.09432318 0.06615569 0.06330413 0.08255624] 
L_V: [ 0.0525274  -0.06110699 -0.01994774  0.00501661 -0.03717655 -0.11200157
 -0.16435948 -0.01050313  0.08809321  0.00071695]
unsatisfied sample number: 4 / 10
100 Lyapunov Risk= 6.215770244598389
V: [0.41820142 0.5409999  0.79727644 0.3681025  0.8101883  0.7639698
 0.7444578  0.5201716  0.8097011  0.8113744 ] 
L_V: [-0.03275818  0.11352111  0.15692435  0.02038437 -0.06771705  0.34876513
  0.08955294  0.5028398  -0.07557252 -0.05658466]
unsatisfied sample number: 6 / 10
200 Lyapunov Risk= 3.6875855922698975
V: [0.16560881 0.16265887 0.88526976 0.47945842 0.8272452  0.91507107
 0.43549445 0.8926228  0.34511945 0.7897651 ] 
L_V: [0.24701162 0.12413705 0.1179286  0.3975738  0.22080636 0.05126626
 0.26117796 0.09060867 0.7182742  0.61447626]
unsatisfied sample number: 10 / 10
300 Lyapunov Risk= 3.2875163555145264
V: [0.86347586 0.8947055

KeyboardInterrupt: 

In [85]:
model = Net(D_in,H1, D_out)
model.load_state_dict(torch.load('Lyapunov_NN_gym.pth'))
L = []
i = 0 
t = 0
max_iters = 2000
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

while i < 2 and not valid: 
    V_candidate = model(x)
    X0 = model(x_0)
    u, _ = policy_model.predict(x_obs)
    # print(type(x), type(u), type(torch.cat((x,torch.from_numpy(u)), dim=1)))
    f = dynamic_model(torch.cat((x,torch.from_numpy(u)), dim=1))
    # f = f_value(x,u)
    Circle_Tuning = Tune(x)
    # Compute lie derivative of V : L_V = ∑∂V/∂xᵢ*fᵢ
    L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                        *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()),0)

    # With tuning term 
    Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()\
                +2.2*((Circle_Tuning-6*V_candidate).pow(2)).mean()+(X0).pow(2)
    # Without tuning term
#         Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()+ 1.2*(X0).pow(2)
    print(x[:5], u[:5], f[:5], L_V[:5])
    
    print(i, "Lyapunov Risk=",Lyapunov_risk.item())
    L.append(Lyapunov_risk.item())
    optimizer.zero_grad()
    Lyapunov_risk.backward()
    optimizer.step()

    w1 = model.layer1.weight.data.numpy()
    w2 = model.layer2.weight.data.numpy()
    b1 = model.layer1.bias.data.numpy()
    b2 = model.layer2.bias.data.numpy()
    i+=1


tensor([[ 0.5759,  2.6159],
        [-2.3344, -1.7129],
        [ 2.6748, -5.9888],
        [ 0.6697, -0.2034],
        [ 2.8404,  3.5864]]) [[-2.       ]
 [-2.       ]
 [-1.988219 ]
 [-1.9999838]
 [ 2.       ]] tensor([[ 0.7629,  2.6442],
        [-2.3715, -2.3223],
        [ 2.4222, -6.0445],
        [ 0.6482, -0.2884],
        [ 3.0235,  3.9638]], grad_fn=<SliceBackward0>) tensor([0.4476, 0.3885, 0.1300, 0.0636, 0.7706], grad_fn=<SliceBackward0>)
0 Lyapunov Risk= 2.502589225769043
tensor([[ 0.5759,  2.6159],
        [-2.3344, -1.7129],
        [ 2.6748, -5.9888],
        [ 0.6697, -0.2034],
        [ 2.8404,  3.5864]]) [[-2.       ]
 [-2.       ]
 [-1.988219 ]
 [-1.9999838]
 [ 2.       ]] tensor([[ 0.7629,  2.6442],
        [-2.3715, -2.3223],
        [ 2.4222, -6.0445],
        [ 0.6482, -0.2884],
        [ 3.0235,  3.9638]], grad_fn=<SliceBackward0>) tensor([0.4647, 0.4049, 0.1168, 0.0623, 0.7748], grad_fn=<SliceBackward0>)
1 Lyapunov Risk= 2.507005214691162


### Checking result with smaller epsilon ( Lie derivative of V <= epsilon )

In [ ]:
epsilon = -0.00001
start_ = timeit.default_timer() 
result = CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config, epsilon)
stop_ = timeit.default_timer() 

if (result): 
    print("Not a Lyapunov function. Found counterexample: ")
else:  
    print("Satisfy conditions with epsilon= ",epsilon)
    print(V_learn, " is a Lyapunov function.")
t += (stop_ - start_)

('Satisfy conditions with epsilon= ', -1e-05)
(<Expression "tanh((0.49927884340286255 - 0.41210386157035828 * tanh((-0.79895162582397461 - 0.053098957985639572 * x1 + 0.046276744455099106 * x2)) + 0.82691246271133423 * tanh((-0.6677706241607666 + 0.69698750972747803 * x1 - 0.0021623193752020597 * x2)) - 0.9570583701133728 * tanh((0.83614975214004517 + 0.9072798490524292 * x1 + 0.013366221450269222 * x2)) - 0.059784829616546631 * tanh((0.90165179967880249 + 0.16413372755050659 * x1 - 0.36351147294044495 * x2)) + 0.97691076993942261 * tanh((1.1444443464279175 + 0.028182908892631531 * x1 - 0.026499949395656586 * x2)) - 0.27872595191001892 * tanh((1.3038069009780884 - 0.43813130259513855 * x1 - 0.24401682615280151 * x2))))">, ' is a Lyapunov function.')


### More details on Lyapunov risk
Generally, we start training with Lyapunov risk without the tuning term.      
For example, (1* F.relu(-V_candidate)+ 1.5* F.relu(L_V+0.5)).mean()+ 1.2*(X0).pow(2)    
The weight of each term (1, 1.5, 1.2) can be tuned for balancing each Lyapunov condition.     
Furthermore, using F.relu(L_V+0.5) allows the learning procedure to seek a candidate Lyapunov function with more negative Lie derivative.   
Here 0.5 is also a tunable parameter based on your goal.    
In this example, we use Lyapunov risk with tuning term for achieving large ROA     